# Description

This notebook is similar to `30-gls_on_phenotypes-phenomexcan.ipynb` but for eMERGE S-MultiXcan results instead of PhenomeXcan.

Since we don't have partition/clusters of eMERGE results, we selected the same set of LVs from the run on PhenomeXcan (`30-gls_on_phenotypes-phenomexcan.ipynb`); and regarding traits, we take the top 20 traits from each LVs to create a list of trait/LVs pairs to run GLSPhenoplier on.

# Environment variables

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import conf

In [3]:
N_JOBS = conf.GENERAL["N_JOBS"]
display(N_JOBS)

3

In [4]:
%env MKL_NUM_THREADS=$N_JOBS
%env OPEN_BLAS_NUM_THREADS=$N_JOBS
%env NUMEXPR_NUM_THREADS=$N_JOBS
%env OMP_NUM_THREADS=$N_JOBS

env: MKL_NUM_THREADS=3
env: OPEN_BLAS_NUM_THREADS=3
env: NUMEXPR_NUM_THREADS=3
env: OMP_NUM_THREADS=3


# Modules

In [5]:
from pathlib import Path

import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from tqdm import tqdm

from gls import GLSPhenoplier

# Settings

In [6]:
N_TOP_TRAITS_FROM_LV = 20

In [7]:
OUTPUT_DIR = conf.RESULTS["GLS"]
display(OUTPUT_DIR)

OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/gls')

In [8]:
OUTPUT_FILENAME = OUTPUT_DIR / "gls_phenotypes-emerge.pkl"
display(OUTPUT_FILENAME)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/gls/gls_phenotypes-emerge.pkl')

# Load data

## eMERGE traits info

In [9]:
# FIXME: in the future, there will be a specific entry in config for the eMERGE directory that should be replaced here
input_filepath = Path(
    conf.DATA_DIR,
    "emerge",
    "eMERGE_III_PMBB_GSA_v2_2020_phecode_AFR_EUR_cc50_counts_w_dictionary.txt",
).resolve()
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/data/emerge/eMERGE_III_PMBB_GSA_v2_2020_phecode_AFR_EUR_cc50_counts_w_dictionary.txt')

In [10]:
emerge_traits_df = pd.read_csv(
    input_filepath,
    sep="\t",
    dtype={"phecode": str},
    usecols=["phecode", "phenotype", "category"],
)

In [11]:
emerge_traits_df = emerge_traits_df.rename(
    columns={
        "phenotype": "phecode_phenotype",
        "category": "phecode_category",
    }
)

In [12]:
emerge_traits_df.shape

(309, 3)

In [13]:
emerge_traits_df.head()

,phecode,phecode_phenotype,phecode_category
0,008,Intestinal infection,infectious diseases
1,008.5,Bacterial enteritis,infectious diseases
2,008.52,Intestinal infection due to C. difficile,infectious diseases
3,038,Septicemia,infectious diseases
4,038.3,Bacteremia,infectious diseases


## eMERGE to PhenomeXcan maps

In [14]:
# FIXME: in the future, there will be a specific entry in config for the eMERGE directory that should be replaced here
emerge_phenomexcan_maps_filepath = Path(
    conf.DATA_DIR,
    "emerge",
    "phecodes_phenomexcan_maps.tsv",
).resolve()
display(emerge_phenomexcan_maps_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/data/emerge/phecodes_phenomexcan_maps.tsv')

In [15]:
emerge_phenomexcan_maps = pd.read_csv(
    emerge_phenomexcan_maps_filepath, sep="\t", dtype={"phecode": str}
)

In [16]:
emerge_phenomexcan_maps = emerge_phenomexcan_maps.dropna(
    subset=["phecode", "phenomexcan"], how="any"
)

In [17]:
emerge_phenomexcan_maps.shape

(381, 6)

In [18]:
emerge_phenomexcan_maps.head()

,phecode,phecode_phenotype,phecode_category,icd10,efo,phenomexcan
0,008,Intestinal infection,infectious diseases,A09,gastroenteritis AND gastroenteritis,A09-Diagnoses_main_ICD10_A09_Diarrhoea_and_gas...
5,008.5,Bacterial enteritis,infectious diseases,A04,intestinal disease AND bacterial disease,A04-Diagnoses_main_ICD10_A04_Other_bacterial_i...
6,038,Septicemia,infectious diseases,A41,sepsis,20002_1657-Noncancer_illness_code_selfreported...
7,038,Septicemia,infectious diseases,A41,sepsis,A41-Diagnoses_main_ICD10_A41_Other_septicaemia
10,053,Herpes zoster,infectious diseases,B02,herpes zoster,B02-Diagnoses_main_ICD10_B02_Zoster_herpes_zoster


## eMERGE (S-MultiXcan) projection

In [19]:
input_filepath = Path(
    conf.RESULTS["PROJECTIONS_DIR"],
    "projection-emerge-smultixcan-mashr-zscores.pkl",
).resolve()
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/projections/projection-emerge-smultixcan-mashr-zscores.pkl')

In [20]:
emerge_projection = pd.read_pickle(input_filepath)

In [21]:
emerge_projection.shape

(987, 309)

In [22]:
emerge_projection.head()

,292.3,079,741,418.1,280,747,591,426.21,244,070.3,...,427,585.4,345,366.2,420.2,195,250.2,250.1,371,562
LV1,0.055748,-0.019274,0.024491,0.012926,-0.026501,-0.043919,0.025264,-0.053251,0.043269,0.002465,...,-0.031739,0.014192,0.008710,-0.024261,-0.004780,0.057256,0.059702,0.036648,-0.022040,0.034427
LV2,-0.045630,-0.008570,0.040969,-0.016198,-0.032321,-0.017629,0.009238,-0.011634,-0.005854,0.033497,...,0.019678,0.040569,-0.009344,-0.012921,-0.055292,0.003193,0.013844,0.008327,0.004302,0.035154
LV3,0.025341,-0.059962,-0.027354,-0.033864,0.009571,0.023318,-0.003872,0.001095,0.028179,0.003639,...,0.041022,-0.036422,0.006088,-0.041302,0.017492,-0.039591,-0.000801,0.009127,0.045741,0.004434
LV4,-0.039736,0.013238,-0.002819,0.029253,-0.029388,-0.003176,0.037212,-0.022370,0.003168,0.039998,...,-0.029197,0.078351,0.044476,-0.027300,0.005332,0.028581,0.004288,0.015797,-0.074106,-0.020936
LV5,-0.012250,-0.002846,-0.025161,0.013880,0.009457,0.024885,0.001274,0.064774,0.039294,-0.000256,...,0.032792,-0.035128,-0.011787,-0.006511,-0.035433,-0.008735,-0.011139,-0.036651,0.003044,0.006505


## eMERGE (S-MultiXcan)

In [23]:
# FIXME: in the future, there will be a specific entry in config for the eMERGE directory that should be replaced here
emerge_smultixcan_zscores_filepath = Path(
    conf.DATA_DIR,
    "emerge",
    "gene_assoc",
    "emerge-smultixcan-mashr-zscores.pkl",
).resolve()

display(emerge_smultixcan_zscores_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/data/emerge/gene_assoc/emerge-smultixcan-mashr-zscores.pkl')

In [24]:
_tmp = pd.read_pickle(emerge_smultixcan_zscores_filepath)

In [25]:
_tmp.shape

(22198, 309)

In [26]:
_tmp.head()

,292.3,079,741,418.1,280,747,591,426.21,244,070.3,...,427,585.4,345,366.2,420.2,195,250.2,250.1,371,562
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,1.043436,1.134028,1.595979,0.399451,0.725100,1.655468,1.803598,0.125250,1.053218,0.922592,...,0.364789,0.453873,0.759558,1.173671,0.298944,1.029646,0.826238,0.685711,2.069701,1.544766
ENSG00000000457,1.236752,0.119837,0.224120,0.200610,1.304190,0.541478,0.646474,0.493759,0.189680,1.563273,...,1.519500,0.201290,1.020996,0.592006,0.632661,0.995453,0.069362,0.460979,0.217046,1.350475
ENSG00000000460,0.497108,0.005690,0.317710,0.184918,1.136574,1.708245,0.288962,1.070026,0.403510,1.106321,...,0.678253,1.166204,1.138126,0.608565,0.449551,0.096689,0.887467,0.143815,0.236647,1.048859
ENSG00000000938,0.374442,1.099899,1.975795,0.107667,0.903520,0.673819,0.855749,0.499088,0.375406,0.815757,...,1.585687,0.584143,0.751384,2.758230,0.572712,0.141023,1.437022,0.288943,0.665727,1.225229
ENSG00000000971,1.108762,1.133163,0.227188,0.222948,0.072709,0.256124,0.540911,0.606947,0.229133,1.557433,...,2.349373,1.122458,0.155346,0.990123,0.326914,1.100210,1.099537,2.511358,0.950262,1.467305


## GLS results on PhenomeXcan

Read results obtained with `30-gls_on_phenotypes.ipynb` (PhenomeXcan)

In [27]:
input_filepath = conf.RESULTS["GLS"] / "gls_phenotypes-phenomexcan.pkl"
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/gls/gls_phenotypes-phenomexcan.pkl')

In [28]:
gls_phenomexcan = pd.read_pickle(input_filepath)

In [29]:
gls_phenomexcan.shape

(2640, 9)

In [30]:
gls_phenomexcan.head()

,part_k,cluster_id,phenotype,lv,lv_with_pathway,coef,pvalue,pvalue_twosided,summary
0,29,17,20003_1140861958-Treatmentmedication_code_simv...,LV744,False,0.017663,0.083346,0.166692,GLS Regression Res...
1,29,17,20003_1140861958-Treatmentmedication_code_simv...,LV841,False,0.033055,0.004121,0.008241,GLS Regression Res...
2,29,17,20003_1140861958-Treatmentmedication_code_simv...,LV65,False,0.013617,0.138593,0.277185,GLS Regression Res...
3,29,17,20003_1140861958-Treatmentmedication_code_simv...,LV515,False,-0.012351,0.834867,0.330265,GLS Regression Res...
4,29,17,20003_1140861958-Treatmentmedication_code_simv...,LV847,True,0.005045,0.346233,0.692465,GLS Regression Res...


## MultiPLIER summary

In [31]:
multiplier_model_summary = pd.read_pickle(conf.MULTIPLIER["MODEL_SUMMARY_FILE"])

In [32]:
multiplier_model_summary.shape

(2157, 5)

In [33]:
multiplier_model_summary.head()

,pathway,LV index,AUC,p-value,FDR
1,KEGG_LYSINE_DEGRADATION,1,0.388059,0.866078,0.956005
2,REACTOME_MRNA_SPLICING,1,0.733057,0.000048,0.000582
3,MIPS_NOP56P_ASSOCIATED_PRE_RRNA_COMPLEX,1,0.680555,0.001628,0.011366
4,KEGG_DNA_REPLICATION,1,0.549473,0.312155,0.539951
5,PID_MYC_ACTIVPATHWAY,1,0.639303,0.021702,0.083739


In [34]:
well_aligned_lvs = multiplier_model_summary[
    (multiplier_model_summary["FDR"] < 0.05) | (multiplier_model_summary["AUC"] >= 0.75)
]

display(well_aligned_lvs.shape)
display(well_aligned_lvs.head())

(469, 5)

,pathway,LV index,AUC,p-value,FDR
2,REACTOME_MRNA_SPLICING,1,0.733057,4.772691e-05,5.816211e-04
3,MIPS_NOP56P_ASSOCIATED_PRE_RRNA_COMPLEX,1,0.680555,1.628217e-03,1.136590e-02
8,REACTOME_MITOTIC_G1_G1_S_PHASES,1,0.686170,2.517619e-04,2.392292e-03
9,IRIS_Monocyte-Day0,2,0.890036,4.315812e-25,1.329887e-22
10,DMAP_MONO2,2,0.904676,1.313970e-16,1.574574e-14


In [35]:
well_aligned_lv_codes = set([f"LV{lvi}" for lvi in well_aligned_lvs["LV index"]])

In [36]:
len(well_aligned_lv_codes)

200

In [37]:
list(well_aligned_lv_codes)[:5]

['LV12', 'LV541', 'LV71', 'LV823', 'LV83']

# Select LVs from previous GLS run on PhenomeXcan

In [38]:
gls_phenomexcan_lvs = gls_phenomexcan["lv"].unique()

In [39]:
gls_phenomexcan_lvs.shape

(188,)

In [40]:
gls_phenomexcan_lvs

array(['LV744', 'LV841', 'LV65', 'LV515', 'LV847', 'LV516', 'LV455',
       'LV381', 'LV575', 'LV330', 'LV530', 'LV91', 'LV797', 'LV367',
       'LV514', 'LV147', 'LV289', 'LV42', 'LV224', 'LV257', 'LV864',
       'LV983', 'LV155', 'LV54', 'LV942', 'LV984', 'LV948', 'LV271',
       'LV605', 'LV453', 'LV610', 'LV987', 'LV913', 'LV568', 'LV342',
       'LV425', 'LV11', 'LV502', 'LV57', 'LV844', 'LV403', 'LV301',
       'LV338', 'LV51', 'LV277', 'LV5', 'LV613', 'LV856', 'LV307',
       'LV877', 'LV851', 'LV954', 'LV808', 'LV510', 'LV387', 'LV443',
       'LV270', 'LV391', 'LV238', 'LV591', 'LV193', 'LV207', 'LV189',
       'LV854', 'LV70', 'LV32', 'LV504', 'LV535', 'LV427', 'LV210',
       'LV174', 'LV861', 'LV769', 'LV787', 'LV56', 'LV154', 'LV603',
       'LV251', 'LV953', 'LV506', 'LV402', 'LV918', 'LV879', 'LV117',
       'LV418', 'LV359', 'LV874', 'LV587', 'LV294', 'LV24', 'LV237',
       'LV835', 'LV171', 'LV452', 'LV904', 'LV241', 'LV300', 'LV50',
       'LV94', 'LV484', 'LV671', '

# Select eMERGE traits

~This is an attempt to first get those eMERGE traits that correspond to the same EFO code in PhenomeXcan.
However, turns out that we only have a few matchings. That's why I'm taking the top 20 traits from each LVs in addition
to these ones.~

In [41]:
# gls_phenomexcan_traits = gls_phenomexcan["phenotype"].unique()

In [42]:
# gls_phenomexcan_traits.shape

In [43]:
# gls_phenomexcan_in_emerge = emerge_phenomexcan_maps[
#     (emerge_phenomexcan_maps["efo"].isin(gls_phenomexcan_traits))
#     | (emerge_phenomexcan_maps["phenomexcan"].isin(gls_phenomexcan_traits))
# ]

In [44]:
# gls_phenomexcan_in_emerge

In [45]:
# gls_emerge_phecodes = gls_phenomexcan_in_emerge["phecode"].unique().tolist()

In [46]:
# these are the mapped traits from PhenomeXcan to phecodes
# gls_emerge_phecodes

# GLSPhenoplier

## Get list of phenotypes/lvs pairs

In [47]:
phenotypes_lvs_pairs = []

# for each LV that was run on PhenomeXcan, I take the top `N_TOP_TRAITS_FROM_LV` traits
# in eMERGE
# ~~~+ global mapped phenotypes (variable `gls_emerge_phecodes`)~~~
for lv in gls_phenomexcan_lvs:
    lv_traits = emerge_projection.loc[lv]
    lv_traits = lv_traits[lv_traits > 0.0]
    lv_traits = lv_traits.sort_values(ascending=False).head(N_TOP_TRAITS_FROM_LV)

    for phenotype_code in set(lv_traits.index.tolist()):  # + gls_emerge_phecodes):
        phenotypes_lvs_pairs.append(
            {
                "phenotype": phenotype_code,
                "lv": lv,
            }
        )

phenotypes_lvs_pairs = pd.DataFrame(phenotypes_lvs_pairs).drop_duplicates()

In [48]:
phenotypes_lvs_pairs = phenotypes_lvs_pairs.sort_values("phenotype").reset_index(
    drop=True
)

In [49]:
phenotypes_lvs_pairs.shape

(3760, 2)

In [50]:
phenotypes_lvs_pairs.head()

,phenotype,lv
0,008,LV18
1,008,LV257
2,008,LV770
3,008,LV514
4,008,LV32


## Run

In [51]:
results = []

pbar = tqdm(total=phenotypes_lvs_pairs.shape[0])

for idx, row in phenotypes_lvs_pairs.iterrows():
    phenotype_code = row["phenotype"]
    lv_code = row["lv"]

    pbar.set_description(f"{phenotype_code} - {lv_code}")

    gls_model = GLSPhenoplier(
        smultixcan_result_set_filepath=emerge_smultixcan_zscores_filepath
    )
    gls_model.fit_named(lv_code, phenotype_code)
    res = gls_model.results

    results.append(
        {
            "phenotype": phenotype_code,
            "lv": lv_code,
            "lv_with_pathway": lv_code in well_aligned_lv_codes,
            "coef": res.params.loc["lv"],
            "pvalue": res.pvalues_onesided.loc["lv"],
            "pvalue_twosided": res.pvalues.loc["lv"],
            "summary": gls_model.results_summary,
        }
    )

    # save results every 10 models trained
    if (idx % 10) == 0:
        pd.DataFrame(results).to_pickle(OUTPUT_FILENAME)

    pbar.update(1)

pbar.close()

796 - LV510: 100%|██████████| 3760/3760 [9:39:46<00:00,  9.25s/it]


In [52]:
results = pd.DataFrame(results)

In [53]:
results.shape

(3760, 7)

In [54]:
results.head()

,phenotype,lv,lv_with_pathway,coef,pvalue,pvalue_twosided,summary
0,008,LV18,True,0.024829,0.021613,0.043225,GLS Regression Res...
1,008,LV257,True,0.029813,0.010222,0.020443,GLS Regression Res...
2,008,LV770,True,0.022392,0.038518,0.077036,GLS Regression Res...
3,008,LV514,False,0.027276,0.015994,0.031988,GLS Regression Res...
4,008,LV32,True,0.040773,0.000721,0.001443,GLS Regression Res...


In [55]:
results.sort_values("pvalue").head(10)

,phenotype,lv,lv_with_pathway,coef,pvalue,pvalue_twosided,summary
449,250.1,LV54,False,0.105369,2.184681e-17,4.369361e-17,GLS Regression Res...
3102,585.32,LV45,True,0.101602,4.446426e-16,8.892851e-16,GLS Regression Res...
440,250.1,LV425,False,0.093755,2.053420e-14,4.106841e-14,GLS Regression Res...
2267,443,LV45,True,0.091128,2.727165e-13,5.454329e-13,GLS Regression Res...
404,244.4,LV57,True,0.089947,2.650956e-12,5.301912e-12,GLS Regression Res...
3461,714.1,LV425,False,0.086154,2.812428e-12,5.624855e-12,GLS Regression Res...
376,244,LV57,True,0.089541,2.974966e-12,5.949932e-12,GLS Regression Res...
701,274.1,LV45,True,0.085661,4.893705e-12,9.787409e-12,GLS Regression Res...
3463,714.1,LV844,True,0.088236,7.920119e-12,1.584024e-11,GLS Regression Res...
3092,585.32,LV847,True,0.085519,2.009862e-11,4.019723e-11,GLS Regression Res...


## Save

In [56]:
results.to_pickle(OUTPUT_FILENAME)